In [1]:
import spacy
import scispacy
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker
from scispacy.hyponym_detector import HyponymDetector
import pandas as pd
from spacy import displacy
import en_core_sci_sm
import deplacy
import networkx as nx
import matplotlib.pyplot as plt

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/spacy/util.py:762: UserWarning: [W095] Model 'en_core_sci_sm' (0.4.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.1.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
with open ("/Users/abhinavshinow/Documents/GitHub/Bert_Based_NER/Data_Collection/heart-data.txt","r") as f:
    text=f.read().replace("\n\n"," ").replace("\n"," ")
text=text[:10000]

In [3]:
#loading all the pretrained scispacy models and adding necassary piplelines
#nlp_core=spacy.load('en_core_sci_sm')
nlp_jnlpba=spacy.load('en_ner_jnlpba_md')
nlp_jnlpba.add_pipe("abbreviation_detector")
nlp_jnlpba.add_pipe("hyponym_detector", last=True, config={"extended": False})
nlp_craft=spacy.load('en_ner_craft_md')
nlp_craft.add_pipe("abbreviation_detector")
nlp_craft.add_pipe("hyponym_detector", last=True, config={"extended": False})
nlp_bionlp=spacy.load('en_ner_bionlp13cg_md')
nlp_bionlp.add_pipe("abbreviation_detector")
nlp_bionlp.add_pipe("hyponym_detector", last=True, config={"extended": False})
nlp_bc5cdr=spacy.load('en_ner_bc5cdr_md')
nlp_bc5cdr.add_pipe("abbreviation_detector")
nlp_bc5cdr.add_pipe("hyponym_detector", last=True, config={"extended": False})

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/spacy/util.py:762: UserWarning: [W095] Model 'en_ner_jnlpba_md' (0.4.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.1.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/spacy/util.py:762: UserWarning: [W095] Model 'en_ner_craft_md' (0.4.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.1.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/opt/homebrew/Caskroom/miniforge/base/e

In [4]:
#Ner models
doc1=nlp_jnlpba(text)
doc2=nlp_craft(text)
doc3=nlp_bionlp(text)
doc4=nlp_bc5cdr(text)

In [5]:
for ents in doc1.ents:
    print(ents)

lymphocyte and monocyte types
white blood cells
pathologically activated eosinophilic white blood cells
psychosocial factors
poverty
insulin


In [6]:
html=displacy.render(doc1,style="ent")

In [7]:
html=displacy.render(doc2,style="ent")

In [8]:
html=displacy.render(doc3,style="ent")

In [9]:
html=displacy.render(doc4,style="ent")

In [10]:
#dependency parsing
for token in doc1:
    print(token.text,token.pos_)

" PUNCT
Cardiovascular ADJ
disease NOUN
( PUNCT
CVD NOUN
) PUNCT
is VERB
a DET
class NOUN
of ADP
diseases NOUN
that DET
involve VERB
the DET
heart NOUN
or CCONJ
blood NOUN
vessels NOUN
. PUNCT
CVD NOUN
includes VERB
coronary ADJ
artery NOUN
diseases NOUN
( PUNCT
CAD NOUN
) PUNCT
such ADJ
as ADP
angina ADJ
and CCONJ
myocardial ADJ
infarction NOUN
( PUNCT
commonly ADV
known VERB
as ADP
a DET
heart NOUN
attack NOUN
) PUNCT
. PUNCT
Other ADJ
CVDs NOUN
include VERB
stroke NOUN
, PUNCT
heart NOUN
failure NOUN
, PUNCT
hypertensive ADJ
heart NOUN
disease NOUN
, PUNCT
rheumatic ADJ
heart NOUN
disease NOUN
, PUNCT
cardiomyopathy NOUN
, PUNCT
abnormal ADJ
heart NOUN
rhythms NOUN
, PUNCT
congenital ADJ
heart NOUN
disease NOUN
, PUNCT
valvular ADJ
heart NOUN
disease NOUN
, PUNCT
carditis NOUN
, PUNCT
aortic ADJ
aneurysms NOUN
, PUNCT
peripheral ADJ
artery NOUN
disease NOUN
, PUNCT
thromboembolic NOUN
disease NOUN
, PUNCT
and CCONJ
venous ADJ
thrombosis NOUN
. PUNCT
The DET
underlying VERB
mechanism

In [11]:
entity=[]
lemma=[]
label=[]
docs=[doc1,doc2,doc3,doc4]

#Enitities in core model
for doc in docs:
    for ents in doc.ents:
        entity.append(str(ents))
        lemma.append(ents.lemma_)
        label.append(ents.label_)

In [12]:
df = pd.DataFrame({'Entity': entity,'label':label})

In [13]:
df

,Entity,label
0,lymphocyte and monocyte types,CELL_TYPE
1,white blood cells,CELL_TYPE
2,pathologically activated eosinophilic white bl...,CELL_TYPE
3,psychosocial factors,PROTEIN
4,poverty,PROTEIN
...,...,...
347,diabetes mellitus,DISEASE
348,weight loss,DISEASE
349,glucose,CHEMICAL
350,salt,CHEMICAL


In [14]:
df.to_csv('Entity-Labels.csv', header=False, index=False)

In [16]:
def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp_bionlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

In [21]:
entity_pairs=[]
sentences=[]
for sent in doc3.sents:
    sentences.append(str(sent))
for sent in sentences:
    entity_pairs.append(get_entities(sent))

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/scispacy/abbreviation.py:216: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


In [ ]:
#try and use entity-linker to merge the data and find triples
